# Data exploration
This notebook is used to navigate the different CSV file and understand how the data is structured and find whether entries in a column are unique or repeated. This is used to determine which fields can be used as primary keys and for which tables composite keys may be required. It is also used to determine the appropriate datatypes.

**NOTE**: while this code is useful to automatically extract information about the data, the head of the DataFrame are displayed to allow the user to visually check the data and make educated decisions. This is especially useful when it comes to date that this code does not yet handled automatically.

In [50]:
# Dependencies
import pandas as pd
from pathlib import Path
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype

## Exploration function
The function below is used to analyse the data in the files. It looks at the length of the file (number of rows), the number of columns and whether each column can be used as a primary key by comparing the number of unique values to the number or rows in the file. It also determine whether the values are numerical or strings. In case of strings, it looks at the maximum number of character to determine what the maximum VARCHAR length should be in the database.

The function return a DataFrame containing the CSV file data for further analysis.

In [74]:
def explore_data(filename):
    # Print file name
    print(f"File: {filename}")

    # Load the CSV file into a DataFrame
    csvfile = Path(f'data/{filename}')
    df = pd.read_csv(csvfile)

    # Size of file
    print(f"Length of file: {len(df)} rows")

    # Number of columns
    num_col = len(df.columns)
    print(f"Number of columns: {num_col}")

    for i in range(num_col):
        # Count the number of unique values in column i
        output_string = f"- Column {i}: '{df.columns[i]}' has {df[df.columns[i]].nunique()} unique values"
    
        # If the number of unique value in column i is the same as the number of rows...
        if df[df.columns[i]].nunique() == len(df):
            # ... then the column can be used as a primary key
            output_string += ', and can be used as primary key'
    
        # Check whether the data is numeric
        if is_numeric_dtype(df[df.columns[i]]):
            # The code will suggest to use INT for any numeric value. This must be checked by the user.
            output_string += '. Values are numeric. Suggest: INT'
        
        # Check whether the data are characters
        elif is_string_dtype(df[df.columns[i]]):

            max_length = df[df.columns[i]].map(lambda x: len(x)).max()

            if max_length == 1:
                # If there is only one charachter, CHAR(1) is suggested as data type.
                output_string += f". Values are strings. Suggest: CHAR(1)"
            else:
                # If the maximum number of character is greater than one, VARCHAR(max) is suggested, a larger number may be used.
                output_string += f". Values are strings. Suggest: VARCHAR({max_length})"

        # State that the datatype cannot be determine
        else:
            output_string += '. Values are of an unknown type.'

        # Print the information about column i
        print(output_string)

    print('Extract of data:')
    display(df.head(2))
    return df

## File 1: departments.csv

In [75]:
filename = 'departments.csv'
df = explore_data(filename)

File: departments.csv
Length of file: 9 rows
Number of columns: 2
- Column 0: 'dept_no' has 9 unique values, and can be used as primary key. Values are strings. Suggest: VARCHAR(4)
- Column 1: 'dept_name' has 9 unique values, and can be used as primary key. Values are strings. Suggest: VARCHAR(18)
Extract of data:


,dept_no,dept_name
0,d001,Marketing
1,d002,Finance


## File 2: dept_emp.csv

In [76]:
filename = 'dept_emp.csv'
df = explore_data(filename)

File: dept_emp.csv
Length of file: 331603 rows
Number of columns: 2
- Column 0: 'emp_no' has 300024 unique values. Values are numeric. Suggest: INT
- Column 1: 'dept_no' has 9 unique values. Values are strings. Suggest: VARCHAR(4)
Extract of data:


,emp_no,dept_no
0,10001,d005
1,10002,d007


## File 3: dept_manager.csv

In [77]:
filename = 'dept_manager.csv'
df = explore_data(filename)

File: dept_manager.csv
Length of file: 24 rows
Number of columns: 2
- Column 0: 'dept_no' has 9 unique values. Values are strings. Suggest: VARCHAR(4)
- Column 1: 'emp_no' has 24 unique values, and can be used as primary key. Values are numeric. Suggest: INT
Extract of data:


,dept_no,emp_no
0,d001,110022
1,d001,110039


## File 4: employees.csv

In [78]:
filename = 'employees.csv'
df = explore_data(filename)

File: employees.csv
Length of file: 300024 rows
Number of columns: 7
- Column 0: 'emp_no' has 300024 unique values, and can be used as primary key. Values are numeric. Suggest: INT
- Column 1: 'emp_title_id' has 7 unique values. Values are strings. Suggest: VARCHAR(5)
- Column 2: 'birth_date' has 4750 unique values. Values are strings. Suggest: VARCHAR(10)
- Column 3: 'first_name' has 1276 unique values. Values are strings. Suggest: VARCHAR(14)
- Column 4: 'last_name' has 1638 unique values. Values are strings. Suggest: VARCHAR(16)
- Column 5: 'sex' has 2 unique values. Values are strings. Suggest: CHAR(1)
- Column 6: 'hire_date' has 5434 unique values. Values are strings. Suggest: VARCHAR(10)
Extract of data:


,emp_no,emp_title_id,birth_date,first_name,last_name,sex,hire_date
0,473302,s0001,7/25/1953,Hideyuki,Zallocco,M,4/28/1990
1,475053,e0002,11/18/1954,Byong,Delgrande,F,9/7/1991


## File 5: salaries.csv

In [79]:
filename = 'salaries.csv'
df = explore_data(filename)

File: salaries.csv
Length of file: 300024 rows
Number of columns: 2
- Column 0: 'emp_no' has 300024 unique values, and can be used as primary key. Values are numeric. Suggest: INT
- Column 1: 'salary' has 50355 unique values. Values are numeric. Suggest: INT
Extract of data:


,emp_no,salary
0,10001,60117
1,10002,65828


## File 6: titles.csv

In [80]:
filename = 'titles.csv'
df = explore_data(filename)

File: titles.csv
Length of file: 7 rows
Number of columns: 2
- Column 0: 'title_id' has 7 unique values, and can be used as primary key. Values are strings. Suggest: VARCHAR(5)
- Column 1: 'title' has 7 unique values, and can be used as primary key. Values are strings. Suggest: VARCHAR(18)
Extract of data:


,title_id,title
0,s0001,Staff
1,s0002,Senior Staff


# Conclusions from the exploration
## Conclusions from File 1
1. Column `dept_no` should be used as PRIMARY KEY.
2. No FOREIGN KEY are used.

## Conclusion from File 2
1. Column `emp_no` has repeating values. An employee can therefore be part of multiple departments.
2. Column `dept_no` has repeating values. A depeartment can therefore include multiple employees.
3. No PRIMARY KEY can be defined with the existing values.

## Conclusions from File 3
1. Column `dept_no` has repeating values. A department may therefore have multiple managers.
2. Column `emp_no` has no repeating values. An employee can then be manager in only one department.

## Conclusions from File 4
1. Column `emp_no` should be used as PRIMARY KEY.
2. All the other columns have repeating values.
3. Columns `birth_date` and `hire_date` are in format MDY (US)

## Conclusions from File 5
1. Column `emp_no` has no repeating values. Each employee is therefore paid only one salary (although they may be part of multiple departments.)
2. Column `salary` has repeating values. Employees may be on the same salary.

## Conclusions from File 6
1. Column `title_id` should be used as PRIMARY KEY.
2. Column `title` has no repeating values.

# Quick Database Diagram
The following code can be used on https://app.quickdatabasediagrams.com to create the data model.